In [ ]:
import os
os.sys.path.insert(0, "..")

import matplotlib.pyplot as plt
import numpy as np
import pystan
import seaborn as sns
import stravalib
from secret import *

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 7)

# Modelling my strava data

## 1. Get the data out of the API

In [ ]:
client = stravalib.Client(access_token=ACCESS_TOKEN)
client.get_athlete()
activities = client.get_activities()
runs = (ai for ai in activities if ai.type == "Run" and ai.manual == False)

def get_run_data(run):
    streams = client.get_activity_streams(
        run.id,
        types=["time", "distance", "grade_smooth", "velocity_smooth", "moving"]
    )
    m = streams["moving"].data
    data = {
        "time": np.array(streams["time"].data)[m],
        "distance": np.array(streams["time"].data)[m],
        "speed": np.array(streams["velocity_smooth"].data)[m],
        "gradient": np.array(streams["grade_smooth"].data)[m]
    }
    return data
data = get_run_data(next(runs))

In [ ]:
plt.plot(data["time"], np.log1p(data["speed"]))

# 2. Simple model

A mean reverting random walk (Ornstein-Uhlenbeck process):

$$u_i \sim \mathcal{N}\left( u_{i-1} + \theta(\mu_{i-1} - u_{i - 1}),\;\sigma^2\right)$$

where $u_i = \log(1 + v_i)$.

In [ ]:
simple_model = """
data {
  int<lower=0> n;
  vector[n] log_v;
}
parameters {
  // random walk parameters
  real<lower=0, upper=1> theta;
  real<lower=0> sigma;
  real mu;
}
model {
    for (i in 2:n) log_v[i] ~ normal(log_v[i - 1] + theta*(mu - log_v[i - 1]), sigma);
}
generated quantities {
  vector[n] log_v_rep;
  log_v_rep[1] = normal_rng(mu, sigma);
  for (i in 2:n) log_v_rep[i] = normal_rng(log_v_rep[i - 1] + theta*(mu - log_v_rep[i - 1]), sigma);
}
"""

In [ ]:
model = pystan.StanModel(model_code=simple_model)

In [ ]:
stan_data = {
    "n": len(data["speed"]),
    "log_v": np.log1p(data["speed"])
}
fit = model.sampling(data=stan_data, seed=42)

In [ ]:
plt.plot(fit["log_v_rep"][0, :])
plt.plot(np.log1p(data["speed"]));

This model is overestimating the variance in speed - presumably because there are some outlier points. These might be because I had to stop to cross the road, or just an issue with the GPS.

## 3. Model with outliers

$$u_i \sim \mathcal{N}\left( u_{i-1} + \theta(\mu_{i-1} - u_{i - 1}) + p_i,\;\sigma^2\right)$$

where $u_i = \log(1 + v_i)$ and $p_i$ is constant that allows for extreme values. Since there is one of these per point in the timeseries, but I only expect a few of them to be non-zero, a prior that induces sparseness is appropriate. The usual L1 (LASSO or Laplace) prior isn't appropriate when sampling from the full posterior, so use a [Finnish Horseshoe](https://arxiv.org/abs/1707.01694). Later it might turn out that sampling isn't really needed, in which case LASSO should be ok.

In [ ]:
outlier_model = """
data {
  int<lower=0> n;
  vector[n] log_v;
}
transformed data {
  real m0 = 10; 
  real slab_scale = 3; 
  real slab_scale2 = square(slab_scale);
  real slab_df = 25;
  real half_slab_df = 0.5 * slab_df;
}
parameters {
  // random walk parameters
  real<lower=0, upper=1> theta;
  real<lower=0> sigma;
  real<lower=0> mu;
  
  //outlier effects + finnish horseshoe prior
  vector[n - 1] p_tilde;
  vector<lower=0>[n - 1] lambda;
  real<lower=0> tau_tilde;
  real<lower=0> c2_tilde;
}
transformed parameters {
  vector[n - 1] p;
  {
    real tau0 = (m0 / (n - m0)) * (sigma / sqrt(1.0 * n));
    real c2 = slab_scale2 * c2_tilde;
    real tau = tau0 * tau_tilde;
    vector[n - 1] lambda_tilde =
      sqrt( c2 * square(lambda) ./ (c2 + square(tau) * square(lambda)) );
    p = tau * lambda_tilde .* p_tilde;
  }
}
model {
    for (i in 2:n) log_v[i] ~ normal(log_v[i - 1] + theta*(mu - log_v[i - 1]) + p[i - 1], sigma);
    sigma ~ normal(0, 1);
    mu ~ normal(1, 2);
    p_tilde ~ normal(0, 1);
    lambda ~ cauchy(0, 1);
    c2_tilde ~ inv_gamma(half_slab_df, half_slab_df);
    tau_tilde ~ cauchy(0, 1);
}
generated quantities {
  vector[n] log_v_rep;
  log_v_rep[1] = log_v[1];
  for (i in 2:n) log_v_rep[i] = normal_rng(log_v_rep[i - 1] + theta*(mu - log_v_rep[i - 1]) + p[i - 1], sigma);
}
"""

In [ ]:
model = pystan.StanModel(model_code=outlier_model)

In [ ]:
stan_data = {
    "n": len(data["speed"]),
    "log_v": np.log1p(data["speed"])
}
fit = model.sampling(data=stan_data, seed=42)

In [ ]:
plt.plot(fit["log_v_rep"][1, :])
plt.plot(np.log1p(data["speed"]));

In [ ]:
sns.distplot(fit["log_v_rep"][2, :])
sns.distplot(np.log1p(data["speed"]));

That's a bit better, although the sparse prior might not be forcing the parameters to zero often enough as there are hints of overfitting.

In [ ]:
fit.plot(["p"]);

Starting to think this might be the wrong model actually...